# Pedersen Commitments

- *Non-Interactive and Information-Theoretic Secure Verifiable Secret Sharing* : https://doi.org/10.1007/3-540-46766-1_9 

P.S : I'll trust you'll sci-hub your way to a digital copy.

Pedersen Commitments are a way to commit to a secret value to divulge it later, originally it was used to construct a k-n [Verifiable Secret Sharing] scheme, a **VSS** is a scheme where n-people share a secret with each other such as a k-group from n can reconstruct the secret from their shares.

While similar to *Shamir's secret sharing*. Pedersen scheme is verifiable, and non-interactive. This is a consequence of  his commitment scheme the one we're going to talk about.
Essentially they can each verify the correctness of their shares without interacting with each other.

## Cryptographic Commitments

A Cryptographic Commitment is a protocol to commit to a message $m$ *secretly* such as a verifier can always verifier the commitment later on.

The protocol boils down to the following :

1. Alice picks or is provided a message $m$.
2. Alice picks a *random* $r$ and computs $c = C(m,r)$
3. Alice publishes $c$
4. At some point in the future Alice publishes $(m,r)$
5. Bob who wishes to verify Alice's commitment computes $C(m,r)$ and checks equality with the public value $c$

The protocol holds if $c$ divulges nothing about $m$ (Shannon secrecy), and the probability of finding $(m,r)$ and $(m',r')$ such as $m \neq m'$ and $C(m,r) = C(m',r')$ is **negligible**.

## Pedersen Commitment

A Pedersen Commitment uses a group $(G,.)$ of large order $q$ where the DLP is hard.
Two random generators $g$ and $h$ are choosen, a random secret $r$ is uniformly picked from $\mathbb{Z}_q$ .

The commitment is :

- **$C(m,r) = g^m . h^r$**

## Protocol Parameters

Some references for the ECC based parameters :

- [Pedersen Commitments over Edwards Curves](http://orbilu.uni.lu/bitstream/10993/33705/1/MSPN2017.pdf)
- [Investigation into Confidential Transaction](https://github.com/AdamISZ/ConfidentialTransactionsDoc/blob/master/essayonCT.pdf)

Essentially Elliptic Curves computations are fast, smaller (256-bit vs 2048-bit at the same security level (160 bit I think).

## A Demo

Our demo we'll follow the below steps to generate parameters and make some commitments :D

1. Pick a large prime $q$ and $p$ a large safe prime such as $p = 2q + 1$
2. Let $h$ a randomly picked generator of cyclic group of order $q$.
3. Let $\alpha$ a randomly picked element of $\mathbb{Z}_q$ and compute $g = h^\alpha$
4. Let $r$ be our randomly picked secreal seal value from **$\mathbb{Z}_p$**
5. Let $x$ be our commited value we compute $C(x,r) = h^r . g^x$
6. A Verifier can later verify this given the commitment bases $(h,g)$ and $(x,r)$.

One neat consequence here is that $C(x_1,r_1) . C(x_2,r_2) = C(x_1+x_2,r_1+r_2)$ , the proof is left
as an exercice to the reader ;).


In [96]:
def gengen(p):
    # given a prime p and the prime factorisation of p-1
    # this algorithm outputs a generator of a Zp
    q = prime_divisors(p-1)[-1] # get the largest prime divisor of p-1
    h = randint(1,p-1)
    y = Integer((p-1)/q)
    g = power_mod(h,y,p)
    if g == 1 :
        g = power_mod(h,Integer((p-1)//q),p)
    return g    
def paramgen(l,t) :
    # lt are the security parameters
    q = random_prime(2^l,lbound=2^(l-1))
    p = 2*q + 1
    h = gengen(q)
    alpha = randint(1,q-1)
    g = power_mod(h,alpha,p)
    return (h,g,p,q)

def mul_mod(a,b,p):
    return mod(a*b,p)
        
def commit(x,r,h,g,p,q):
    return mod(power_mod(h,r,p)*power_mod(g,x,p),p)

def open_commitment(c,x,r,h,g,p,q):
    return c == commit(x,r,h,g,p,q)

In [97]:
(h,g,p,q) = paramgen(256,128)

In [98]:
r1 = randint(1,p-1)
r2 = randint(1,p-1)
x1 = 5
x2 = 10
c1 = commit(x1,r1,h,g,p,q)
c2 = commit(x2,r2,h,g,p,q)


In [99]:
assert(open_commitment(c1,x1,r1,h,g,p,q) == True)
assert(open_commitment(c2,x2,r2,h,g,p,q) == True)

In [101]:
assert(mul_mod(c1,c2,p) == commit(x1+x2,r1+r2,h,g,p,q))

- **REFERENCES** :
    - http://cryptonite.info/files/HMBC.pdf
    - http://orbilu.uni.lu/bitstream/10993/33705/1/MSPN2017.pdf    
    - https://github.com/AdamISZ/ConfidentialTransactionsDoc/blob/master/essayonCT.pdf

## NOTE

Doing this for ECC is a matter of translation, operations on curves are defined as addition (P a point on a curve can be added to itself k times that's kP *on an elliptic curves, a public key is basically a random scalar k (Private key) multiplied by a point on the curve*.

I'll write